In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [ ]:
# Load the Public Bins Locations Dataset
# File path for the Public Bins data
public_bins_locations_path = 'Public_Bins_Locations_MappingFCC.csv'  # Update path if necessary

In [ ]:
# Load the dataset
public_bins_locations = pd.read_csv(public_bins_locations_path)

In [ ]:
# Checking if the dataset is loaded correctly
print("Public Bins Locations Columns:", public_bins_locations.columns)

In [ ]:
if 'Cluster' not in public_bins_locations.columns:
    kmeans = KMeans(n_clusters=5, random_state=42)
    public_bins_locations['Cluster'] = kmeans.fit_predict(public_bins_locations[['X', 'Y']])
    print("Cluster column created using KMeans.")
else:
    print("Cluster column already exists.")

In [ ]:
# Preprocess Data
# Features and target
features = ['X', 'Y', 'type']  
target = 'Cluster'  

In [ ]:
public_bins_locations = public_bins_locations.dropna(subset=features + [target])

In [ ]:
#'type' column
label_encoder = LabelEncoder()
public_bins_locations['type_encoded'] = label_encoder.fit_transform(public_bins_locations['type'])

In [ ]:
# Prepare data
X = public_bins_locations[['X', 'Y', 'type_encoded']]
y = public_bins_locations[target]

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train Random Forest Classifier 
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = rf_classifier.predict(X_test)

In [ ]:
# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("\n=== Clustering Prediction: Classification Results ===")
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Plot Feature Importance 
feature_importances = rf_classifier.feature_importances_
plt.figure(figsize=(8, 6))
sns.barplot(x=feature_importances, y=['X', 'Y', 'type_encoded'], palette="viridis")
plt.title("Feature Importance for Cluster Prediction")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
# Load the Brown Bin Data
# File path for the Brown Bin data
brown_bin_data_path = 'brown_bin_data.csv'  

In [ ]:
# Load the dataset
brown_bin_data = pd.read_csv(brown_bin_data_path)

In [ ]:
print("Brown Bin Data Columns:", brown_bin_data.columns)

In [ ]:
# Predictive Model for Brown Bin Data 
# Features and target
features = ['WEEK', 'YEAR', 'Lift count', 'Average weight per bin']
target = 'Brown weight collected'

In [ ]:
# Drop missing values
brown_bin_data = brown_bin_data.dropna(subset=features + [target])

In [ ]:
# Split into train and test
X = brown_bin_data[features]
y = brown_bin_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test)

In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print("\n=== Brown Bin Data: Regression Model Results ===")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.2f}")

In [ ]:
# Feature Importance
feature_importances = rf_model.feature_importances_
plt.figure(figsize=(8, 6))
sns.barplot(x=feature_importances, y=features, palette="viridis")
plt.title("Feature Importance for Brown Weight Collected Prediction")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
# Plot Actual vs. Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6, label="Predictions")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Ideal Fit")
plt.title("Actual vs. Predicted Brown Weight Collected")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.legend()
plt.show()

In [ ]:
# Residual Analysis
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30, color="blue")
plt.title("Residuals Distribution")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(0, color='r', linestyle='--')
plt.title("Residuals vs. Predicted")
plt.xlabel("Predicted")
plt.ylabel("Residuals")
plt.show()

In [ ]:
# Save the model
joblib.dump(rf_model, "brown_bin_rf_model.pkl")